## **PROBLEM**

The Problem of Anomaly Detection is divided into two parts:

1. Given a network, we need a program to find anomalies in that particular network, based on some previous known data [Training Data] - **SUPERVISED LEARNING**

2. Given a network, we need a program to find anomalies in that particular network, without any previous known data [Testing Data] - **UNSUPERVISED LEARNING**

We will use the following dataset for this problem: https://www.kaggle.com/datasets/sampadab17/network-intrusion-detection/code


**Note:** Yes, I looked at the solution notebooks for this problem, but I did not copy, just saw what things people are using and then took all the things for which I could understand what was happening.

**Importing Libraries**

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import RFE
import itertools
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import plotly.express as px
import plotly.graph_objects as go

### **Reading Data**

In [3]:
train=pd.read_csv(r'C:\Users\anmol\Desktop\Coding\moniBakchodi\hackathon\archive\Train_data.csv')

In [4]:
test=pd.read_csv(r'C:\Users\anmol\Desktop\Coding\moniBakchodi\hackathon\archive\Test_data.csv')

In [5]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [6]:
test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71


### **Data Preprocessing**

In [7]:
# Compare columns in train and test
print('Columns in train but not in test:', set(train.columns)-set(test.columns))
print('Columns in test but not in train:', set(test.columns)-set(train.columns))

Columns in train but not in test: {'class'}
Columns in test but not in train: set()


Perfect!

**Checking Null Values**

In [8]:
for i in train.columns:
    # Check Any NA Values
    if train[i].isnull().sum() > 0:
        print(i, train[i].isnull().sum())

In [9]:
for i  in test.columns:
    # Check Any NA Values
    if test[i].isnull().sum() > 0:
        print(i, test[i].isnull().sum())

No Null Values in Train Dataset as well as Test Dataset

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25192 entries, 0 to 25191
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     25192 non-null  int64  
 1   protocol_type                25192 non-null  object 
 2   service                      25192 non-null  object 
 3   flag                         25192 non-null  object 
 4   src_bytes                    25192 non-null  int64  
 5   dst_bytes                    25192 non-null  int64  
 6   land                         25192 non-null  int64  
 7   wrong_fragment               25192 non-null  int64  
 8   urgent                       25192 non-null  int64  
 9   hot                          25192 non-null  int64  
 10  num_failed_logins            25192 non-null  int64  
 11  logged_in                    25192 non-null  int64  
 12  num_compromised              25192 non-null  int64  
 13  root_shell      

**Further Processing**

In [11]:
# Get Numeric Columns
numeric_columns = train.select_dtypes(include=[np.number]).columns
other_columns = train.columns.difference(numeric_columns)

In [12]:
len(numeric_columns), len(other_columns)

(38, 4)

In [13]:
print("NUMERIC COLUMNS:", numeric_columns)
print("OTHER COLUMNS:", other_columns)

NUMERIC COLUMNS: Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate'],
      dtype='object')
OTHER COLUMNS: Index(['class', 'flag', 'protocol_type', 'service'], dtype='object')


In [14]:
output_column = 'class'
other_columns = other_columns.difference([output_column])

In [15]:
numeric_columns, other_columns, output_column

(Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
        'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
        'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
        'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
        'is_guest_login', 'count', 'srv_count', 'serror_rate',
        'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
        'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
        'dst_host_srv_count', 'dst_host_same_srv_rate',
        'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
        'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
        'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
        'dst_host_srv_rerror_rate'],
       dtype='object'),
 Index(['flag', 'protocol_type', 'service'], dtype='object'),
 'class')

**One Hot Encoding**

In [16]:
# Number of Distinct Values in Each Column
for i in other_columns:
    print(i, " -> Number of Distinct Values:", train[i].nunique())

flag  -> Number of Distinct Values: 11
protocol_type  -> Number of Distinct Values: 3
service  -> Number of Distinct Values: 66


In [17]:
train = pd.get_dummies(train, columns=other_columns, drop_first=True)

In [18]:
train.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,service_systat,service_telnet,service_tim_i,service_time,service_urh_i,service_urp_i,service_uucp,service_uucp_path,service_vmnet,service_whois
0,0,491,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,0,146,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,0,232,8153,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,False
4,0,199,420,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,False


In [19]:
train.columns

Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'service_systat', 'service_telnet', 'service_tim_i', 'service_time',
       'service_urh_i', 'service_urp_i', 'service_uucp', 'service_uucp_path',
       'service_vmnet', 'service_whois'],
      dtype='object', length=116)

**TRAIN TEST SPLIT**

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('class', axis=1), train['class'], test_size=0.2, random_state=42, stratify=train['class'])

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20153, 115), (5039, 115), (20153,), (5039,))

**NORMALISATION**

In [22]:
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20153, 115), (5039, 115), (20153,), (5039,))

In [24]:
X_train[0]

array([0.00000000e+00, 7.72839861e-07, 2.47913911e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [25]:
y_train[0]

'normal'

**Visualizing in 3-Dimensions**

In [26]:
# Visualize the Data in 3 dimensions using TSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=42)
X_train_3d_tsne = tsne.fit_transform(X_train)

X_train_3d_tsne.shape

(20153, 3)

In [27]:
colors = ['red' if i == 'normal' else 'blue' for i in y_train]

In [28]:
# Visualize the Data in 3 dimensions via Plotly using TSNE, Mark y_train[i] = 'normal' as red and y_train[i] = 'anomaly' as blue
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=X_train_3d_tsne[:, 0], y=X_train_3d_tsne[:, 1], z=X_train_3d_tsne[:, 2], mode='markers', marker=dict(color=colors)))
fig.show()

Clear Pattern Seems to be Visible.

In [29]:
# Visualize the Data in 2 dimensions using TSNE
tsne = TSNE(n_components=2, random_state=42)
X_train_2d_tsne = tsne.fit_transform(X_train)

X_train_2d_tsne.shape

(20153, 2)

In [30]:
# Visualize the Data in 2 dimensions via Plotly using TSNE, Mark y_train[i] = 'normal' as red and y_train[i] = 'anomaly' as blue
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train_2d_tsne[:, 0], y=X_train_2d_tsne[:, 1], mode='markers', marker=dict(color=colors)))
fig.show()

### **UNSUPERVISED TECHNIQUES BASED ON ALL FEATURES**

We will use 3 techniques for anomaly detection [**Out of which 2 Dont work for this problem**]:

1. **Isolation Forest** : It is an unsupervised learning algorithm that belongs to the ensemble decision trees family. It is based on the Decision Tree algorithm. In these methods, the decision tree is used to isolate each observation. This method is highly useful for detecting anomalies in the data - $Wont$ $work$ as in our data, anomalies are not in minority.

2. **Local Outlier Factor** : The LOF algorithm is an unsupervised outlier detection method which computes the local density deviation of a given data point with respect to its neighbors. It considers as outlier samples that have a substantially lower density than their neighbors - $Wont$ $work$ as in our data, anomalies are not in minority.

3. **DBSCAN** : DBSCAN is a density-based clustering non-parametric algorithm: given a set of points in some space, it groups together points that are closely packed together (points with many nearby neighbors), marking as outliers points that lie alone in low-density regions (whose nearest neighbors are too far away) - $Can$ $work$ as it is based on density and can separate anomalies from normal data.

### **ISOLAION FOREST**

**Spoiler Alert:** Poor Output Below

In [31]:
y_train.value_counts()

class
normal     10759
anomaly     9394
Name: count, dtype: int64

In [32]:
y_test.value_counts()

class
normal     2690
anomaly    2349
Name: count, dtype: int64

In [33]:
# ISOLATION FOREST
from sklearn.ensemble import IsolationForest
isolation_forest = IsolationForest(n_estimators=100, random_state=42)

In [34]:
isolation_forest.fit(X_train)

IsolationForest(random_state=42)

In [35]:
y_pred_train = isolation_forest.predict(X_train)
y_pred_test = isolation_forest.predict(X_test)

In [36]:
y_pred_train.shape, y_pred_test.shape, y_train.shape, y_test.shape

((20153,), (5039,), (20153,), (5039,))

In [37]:
np.unique(y_pred_train, return_counts=True)

(array([-1,  1]), array([   16, 20137], dtype=int64))

In [38]:
# Anomaly Detection
# -1 -> Anomaly
# 1 -> Normal
y_pred_train = np.where(y_pred_train == -1, 'anomaly', 'normal')
y_pred_test = np.where(y_pred_test == -1, 'anomaly', 'normal')

In [39]:
# Classification Report
from sklearn.metrics import classification_report

In [40]:
print("TRAIN CLASSIFICATION REPORT")
print(classification_report(y_train, y_pred_train))

TRAIN CLASSIFICATION REPORT
              precision    recall  f1-score   support

     anomaly       0.38      0.00      0.00      9394
      normal       0.53      1.00      0.70     10759

    accuracy                           0.53     20153
   macro avg       0.45      0.50      0.35     20153
weighted avg       0.46      0.53      0.37     20153



In [41]:
# Classification Report
print("TEST CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred_test))

TEST CLASSIFICATION REPORT
              precision    recall  f1-score   support

     anomaly       0.33      0.00      0.00      2349
      normal       0.53      1.00      0.70      2690

    accuracy                           0.53      5039
   macro avg       0.43      0.50      0.35      5039
weighted avg       0.44      0.53      0.37      5039



In [42]:
# Adjusted Rand Index
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y_train, y_pred_train), adjusted_rand_score(y_test, y_pred_test)

(-3.939612269318777e-05, -8.635531018342895e-05)

**POOR OUTPUT**

### **LOCAL OUTLIER FACTOR**

In [43]:
# LOF
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20)

In [44]:
lof.fit(X_train)

LocalOutlierFactor()

In [45]:
y_pred_train = lof.fit_predict(X_train)
y_pred_test = lof.fit_predict(X_test)

In [46]:
# Unique Values
np.unique(y_pred_train, return_counts=True)

(array([-1,  1]), array([ 3019, 17134], dtype=int64))

In [47]:
# Adjusted Rand Index
adjusted_rand_score(y_train, y_pred_train), adjusted_rand_score(y_test, y_pred_test)

(-0.0008591460201811297, 0.01846259698804876)

In [48]:
# Anomaly Detection
# -1 -> Anomaly
# 1 -> Normal
y_pred_train = np.where(y_pred_train == -1, 'anomaly', 'normal')
y_pred_test = np.where(y_pred_test == -1, 'anomaly', 'normal')

In [49]:
# Classification Report
print("TRAIN CLASSIFICATION REPORT")
print(classification_report(y_train, y_pred_train))

TRAIN CLASSIFICATION REPORT
              precision    recall  f1-score   support

     anomaly       0.45      0.14      0.22      9394
      normal       0.53      0.85      0.65     10759

    accuracy                           0.52     20153
   macro avg       0.49      0.50      0.44     20153
weighted avg       0.49      0.52      0.45     20153



In [50]:
# Classification Report
print("TEST CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred_test))

TEST CLASSIFICATION REPORT
              precision    recall  f1-score   support

     anomaly       0.58      0.30      0.39      2349
      normal       0.57      0.81      0.67      2690

    accuracy                           0.57      5039
   macro avg       0.57      0.55      0.53      5039
weighted avg       0.57      0.57      0.54      5039



**Still POOR**

### **DBSCAN**

In [51]:
# DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)

In [52]:
dbscan.fit(X_train)

DBSCAN()

In [53]:
y_pred_train = dbscan.fit_predict(X_train)
y_pred_test = dbscan.fit_predict(X_test)

In [54]:
# Unique Values
y_pred_train.shape, y_pred_test.shape

((20153,), (5039,))

In [55]:
y_pred_train

array([  0,  -1,   1, ...,   2, 114, 140], dtype=int64)

In [56]:
# Adjusted Rand Index
adjusted_rand_score(y_train, y_pred_train), adjusted_rand_score(y_test, y_pred_test)

(0.17912447916225435, 0.16924404315876476)

**Best of the 3** as we thought

#### **Possible Issue:**

We are treating the problem as anomaly detection, but it is not as 'anomaly' class is not in minority [not a real anomaly] - We should instead take out important features and do **Binary Classification** to get high accuracy on this dataset.

We are using the entire feature set for anomaly detection, which is not a good idea. We should use only the important features for anomaly detection.

To Find Important Features, we can do:

1. **Feature Importance** : We can use Random Forest to find the important features.

2. **PCA** : We can use PCA to find the important features.

3. **Autoencoders** : We can use Autoencoders to find the important features.

Also, Need to remove intercorrelated features.

In [57]:
NUMBER_IMPORTANT_FEATURES_TO_GET = 50

#### **Random Forest Important Features**

In [58]:
# Find Important Features using Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [59]:
random_forest.feature_importances_

array([4.61322736e-03, 1.66919989e-02, 1.45022339e-01, 2.76623620e-05,
       3.75151179e-03, 5.72945601e-05, 8.59240216e-03, 1.77062563e-04,
       5.55588968e-02, 6.51197180e-03, 1.47678801e-04, 4.41925599e-05,
       4.74856928e-04, 2.05856272e-04, 2.38696098e-05, 1.11677097e-04,
       0.00000000e+00, 0.00000000e+00, 8.18602778e-04, 3.13015079e-02,
       1.48596560e-02, 2.00618314e-02, 2.67451789e-02, 1.35120099e-02,
       7.16089187e-03, 5.65915116e-02, 4.30468265e-02, 8.02186267e-03,
       2.96222195e-02, 3.17245640e-02, 6.21373530e-02, 4.79103025e-02,
       3.67355564e-02, 2.07742175e-02, 2.42195391e-02, 4.35032421e-02,
       2.12551676e-02, 1.35812789e-02, 1.44065851e-03, 2.78700775e-04,
       3.69029908e-05, 1.94404115e-03, 1.71282668e-02, 4.11033858e-04,
       6.61864843e-05, 1.95287573e-06, 6.17475565e-02, 9.10828306e-05,
       1.48616495e-02, 1.08034785e-02, 1.02824942e-04, 0.00000000e+00,
       1.41682647e-04, 8.42988646e-10, 3.29815007e-10, 3.36594852e-09,
      

In [60]:
# Pick top - 10
important_features_rfe = random_forest.feature_importances_
important_features_rfe = important_features_rfe.argsort()[::-1][:NUMBER_IMPORTANT_FEATURES_TO_GET]

In [61]:
important_features_rfe, random_forest.feature_importances_[important_features_rfe]

(array([  2,  30,  46,  25,   8,  31,  35,  26,  32,  29,  19,  28,  22,
         71,  34,  36,  33,  21,  62,  42,   1,  48,  20,  95,  37,  23,
         49,  60,  63,   6,  27,  24,   9,  68,   0,   4, 100,  90,  41,
        110,  38,  67,  18, 106,  12,  43,  39,  13,   7,  66], dtype=int64),
 array([0.14502234, 0.06213735, 0.06174756, 0.05659151, 0.0555589 ,
        0.0479103 , 0.04350324, 0.04304683, 0.03673556, 0.03172456,
        0.03130151, 0.02962222, 0.02674518, 0.02661243, 0.02421954,
        0.02125517, 0.02077422, 0.02006183, 0.01849569, 0.01712827,
        0.016692  , 0.01486165, 0.01485966, 0.01440865, 0.01358128,
        0.01351201, 0.01080348, 0.01052371, 0.00979846, 0.0085924 ,
        0.00802186, 0.00716089, 0.00651197, 0.00608485, 0.00461323,
        0.00375151, 0.00288796, 0.00230502, 0.00194404, 0.00160745,
        0.00144066, 0.00094517, 0.0008186 , 0.00055656, 0.00047486,
        0.00041103, 0.0002787 , 0.00020586, 0.00017706, 0.00017608]))

In [62]:
X_train_rfe = X_train[:, important_features_rfe]
X_test_rfe = X_test[:, important_features_rfe]

In [63]:
X_train_rfe.shape, X_test_rfe.shape

((20153, 50), (5039, 50))

#### **PCA Important Features**

In [64]:
# Use PCA for find important features
from sklearn.decomposition import PCA

pca = PCA(n_components=NUMBER_IMPORTANT_FEATURES_TO_GET)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [65]:
X_train_pca.shape

(20153, 50)

In [66]:
# Find Important Features using PCA
pca.explained_variance_ratio_

array([0.42012571, 0.15893223, 0.10702746, 0.04838125, 0.03903922,
       0.02824248, 0.02105838, 0.01691001, 0.01497873, 0.01398697,
       0.01146688, 0.00961659, 0.0068889 , 0.00656151, 0.00593801,
       0.00539958, 0.00465044, 0.0039967 , 0.00379659, 0.00343807,
       0.00326391, 0.00293213, 0.00237433, 0.002225  , 0.00203973,
       0.00188735, 0.00180442, 0.00172593, 0.00160797, 0.00149713,
       0.00148583, 0.00140699, 0.00137432, 0.00137076, 0.00133366,
       0.00130746, 0.0012665 , 0.00123232, 0.00118524, 0.00114082,
       0.00112159, 0.00111013, 0.00110556, 0.0010891 , 0.00106396,
       0.00105443, 0.00104137, 0.00102208, 0.00101144, 0.00099795])

In [67]:
X_train_pca.shape, X_test_pca.shape

((20153, 50), (5039, 50))

#### **Autoencoders Important Features**

In [69]:
X_train.shape, X_test.shape

((20153, 115), (5039, 115))

In [70]:
# Autoencoder Batch Normalization
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization

input_layer = Input(shape=(X_train.shape[1],))
encoded = Dense(100, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dense(50, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dense(25, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
decoded = Dense(50, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(100, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [71]:
autoencoder.fit(X_train, X_train, epochs=100, batch_size=1024, validation_data=(X_test, X_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.2638 - val_loss: 0.2314
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2187 - val_loss: 0.2038
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1842 - val_loss: 0.1490
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1325 - val_loss: 0.0794
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0772 - val_loss: 0.0426
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0486 - val_loss: 0.0310
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0373 - val_loss: 0.0261
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0327 - val_loss: 0.0232
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0276 - val_loss: 0.0215
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0276 - val_loss: 0.0206
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0258 - val_loss: 0.0212
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.

In [72]:
encoder = Model(input_layer, encoded)

X_train_autoencoder = encoder.predict(X_train)
X_test_autoencoder = encoder.predict(X_test)

630/630 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step


In [73]:
X_train_autoencoder.shape, X_test_autoencoder.shape

((20153, 25), (5039, 25))

**NOTE:** We can also try anomaly detection using autoencoders - Reconstruction Error > Threshold => Anomaly as less inputs for anomalies => Model will not be able to reconstruct properly. But it is wont work as 'anomaly' class is not minority [real anomaly].

### PLAN AHEAD:

**Binary Classification** : We can use the important features to do binary classification and get high accuracy on this dataset - Use Random Forest, XGBoost, AdaBoost, ANN, Voting Classifier, etc.

**Above 3 Method with Important Features** : We can use the above 3 methods with important features to get better results - Might not work as we saw above our problem is not really anomaly detection in the data but it is actually **Binary Classification Problem**

